In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 30.7 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 


In [2]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [3]:
from pyomo.environ import *
import numpy as np

In [4]:
coef = np.loadtxt('lab6_practice_coef.txt', delimiter=',')

In [5]:
N = coef.shape[1]-1
M = len(coef)-1

In [6]:
obj_coef = coef[0,:-1]

cons_coef = coef[1:,:-1]
cons_rhs = coef[1:,-1]

In [7]:
cols_x = np.arange(int(N/2))
cols_u = np.arange(int(N/2))

rows = np.arange(M)

In [8]:
m1 = ConcreteModel()
m1.x = Var(cols_x)
m1.u = Var(cols_u,domain=NonNegativeReals)

m1.cons = ConstraintList()

In [9]:
for i in range(M):
    m1.cons.add(sum(cons_coef[i][j]*m1.x[j] + cons_coef[i][j+5]*m1.u[j] for j in cols_x)>=cons_rhs[i])

In [10]:
m1.obj = Objective(expr=summation(m1.u))

In [11]:
solver= SolverFactory('cbc')
opt1 = solver.solve(m1)

### 2)

In [12]:
print('Solver status:', opt1.solver.status)
print('Solver termination condition:',opt1.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


### 3)

In [13]:
# display solution
print('\nObjective = ', m1.obj())

print('\nDecision Variables')
for j in cols_x:
    print('x[',j+1,']:', m1.x[j].value)

for j in cols_u:
    print('u[',j+1,']:', m1.u[j].value)

print('\nConstraints')
m1.cons.display()


Objective =  0.547955725

Decision Variables
x[ 1 ]: -0.046448575
x[ 2 ]: 0.18612441
x[ 3 ]: -0.17131802
x[ 4 ]: 0.0
x[ 5 ]: -0.14406472
u[ 1 ]: 0.046448575
u[ 2 ]: 0.18612441
u[ 3 ]: 0.17131802
u[ 4 ]: 0.0
u[ 5 ]: 0.14406472

Constraints
cons : Size=16
    Key : Lower : Body               : Upper
      1 :   1.0 : 0.9999996250000001 :  None
      2 :   1.0 : 0.9999997599999997 :  None
      3 :   1.0 :  1.458337769999999 :  None
      4 :   1.0 : 1.0000002550000002 :  None
      5 :   1.0 :  1.000000215000001 :  None
      6 :   1.0 : 5.2549776999999995 :  None
      7 :   0.0 :         0.09289715 :  None
      8 :   0.0 :                0.0 :  None
      9 :   0.0 :         0.34263604 :  None
     10 :   0.0 :                0.0 :  None
     11 :   0.0 :         0.28812944 :  None
     12 :   0.0 :                0.0 :  None
     13 :   0.0 :         0.37224882 :  None
     14 :   0.0 :                0.0 :  None
     15 :   0.0 :                0.0 :  None
     16 :   0.0 :        

Constraints 1,2,4,5,8,10,12,14,15,16 are active constraints.
We can also see that the value of each u-i is equal to the absolute value of corrensponding x-i. Hence the solution to our reformulation can also give us the solution to the original linear program. 

In [14]:
coef2 = np.loadtxt('lab6_practice_coef_OP2.txt', delimiter=',')

In [15]:
obj_coef2 = coef2[0,:-1]
cons_coef2 = coef2[1:,:-1]
cons_rhs2 = coef2[1:,-1]

In [16]:
N2 = coef2.shape[1]-1
M2 = len(coef2)-1

cols_a = np.arange(int(N2/2))
cols_b = np.arange(int(N2/2))

rows = np.arange(M2)

In [17]:
m12 = ConcreteModel()
m12.a = Var(cols_a,domain=NonNegativeReals)
m12.b = Var(cols_b,domain=NonNegativeReals)

m12.cons = ConstraintList()
for i in range(M2):
    m12.cons.add(sum(cons_coef2[i][j]*m12.a[j] + cons_coef2[i][j+len(m12.a)]*m12.b[j] for j in cols_a)>=cons_rhs2[i])

m12.obj = Objective(expr=summation(m12.a)+summation(m12.b))    





In [18]:
opt2 = solver.solve(m12)

### 9)

In [19]:
print('Solver status:', opt2.solver.status)
print('Solver termination condition:',opt2.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


### 10)

In [20]:
print('\nObjective = ', m12.obj())

print('\nDecision Variables')
for j in cols_a:
    print('a[',j+1,']:', m12.a[j].value)

for j in cols_b:
    print('b[',j+1,']:', m12.b[j].value)

print('\nConstraints')
m12.cons.display()


Objective =  0.547955725

Decision Variables
a[ 1 ]: 0.0
a[ 2 ]: 0.18612441
a[ 3 ]: 0.0
a[ 4 ]: 0.0
a[ 5 ]: 0.0
b[ 1 ]: 0.046448575
b[ 2 ]: 0.0
b[ 3 ]: 0.17131802
b[ 4 ]: 0.0
b[ 5 ]: 0.14406472

Constraints
cons : Size=6
    Key : Lower : Body               : Upper
      1 :   1.0 : 0.9999996250000001 :  None
      2 :   1.0 : 0.9999997599999997 :  None
      3 :   1.0 :  1.458337769999999 :  None
      4 :   1.0 : 1.0000002550000002 :  None
      5 :   1.0 :  1.000000215000001 :  None
      6 :   1.0 : 5.2549776999999995 :  None


Constraints 1,2,4,5 are active constraints.

### 11)

In [21]:
for i in range(N2//2):
  print(f'Value of x{i+1} is', m12.a[i]()-m12.b[i]())

Value of x1 is -0.046448575
Value of x2 is 0.18612441
Value of x3 is -0.17131802
Value of x4 is 0.0
Value of x5 is -0.14406472


### 12)

In [22]:
print('         OP1',' '*18,'OP2')
for j in cols_x:
    print('x[',j+1,']:','%13.10f'%m1.x[j].value,' '*8 ,'%13.10f'%(m12.a[j]()-m12.b[j]()))

         OP1                    OP2
x[ 1 ]: -0.0464485750          -0.0464485750
x[ 2 ]:  0.1861244100           0.1861244100
x[ 3 ]: -0.1713180200          -0.1713180200
x[ 4 ]:  0.0000000000           0.0000000000
x[ 5 ]: -0.1440647200          -0.1440647200


The value of all Xis are the same in OP1 and OP2. This is because they are equivalent LPs 